In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from portal_analytics.api import AnalyticsApi
import portal_analytics.parser as parser
import portal_analytics.create_rasters as cr
from shapely.geometry import box, Point
import json
import pandas as pd

In [3]:
with open('items.json') as file:
      items = json.load(file)
layers=  items["layers"]
for layer in layers:
    print(layer)

mineral_occurrences
mines
mineral_tenements
scanned_250K_maps
nvcl
all_boreholes


In [4]:
KEY_FILE = 'GeosciencePortal-eeac25f8efb3.json'
ACCOUNT_NAME = 'AUSGIN website'
PROPERTY_NAME = 'AUSGIN'
PROFILE_NAME = 'AUSGIN - exclude internal traffic'


In [5]:
connection=AnalyticsApi(KEY_FILE,ACCOUNT_NAME,PROPERTY_NAME,PROFILE_NAME)

In [6]:
rows, headers = connection.get_results()

In [7]:
raw_results = parser.create_dataframe(rows, headers)
raw_results.to_excel("raw_results.xlsx")

In [8]:
raw_results["ga:eventCategory"].unique()

array(['Add:KnownLayer', 'New:CustomUI', 'New:Downloads',
       'New:Help Panel', 'New:LayerInteraction', 'New:Query'],
      dtype=object)

In [9]:
raw_results["ga:hostname"].unique()

array(['portal.geoscience.gov.au'], dtype=object)

In [14]:
raw_results[raw_results["ga:eventCategory"]=="New:Help Panel"]


,ga:eventCategory,ga:eventAction,ga:eventLabel,ga:hostname,ga:totalEvents
629,New:Help Panel,Click,Download the 1:250K geological map downlod Guide,portal.geoscience.gov.au,19
630,New:Help Panel,Click,Download the portal User Guide,portal.geoscience.gov.au,3
631,New:Help Panel,Click,Help desk contact and other enquiries and feed...,portal.geoscience.gov.au,6


In [ ]:
data_frame = parser.process_dataframe(raw_results)
data_frame["ga:totalEvents"]=data_frame["ga:totalEvents"].apply(int)
add_layers=data_frame[data_frame["ga:eventCategory"]=="Add:KnownLayer"].groupby(by="ga:eventAction")["ga:totalEvents"].sum()

In [ ]:
pd.DataFrame(add_layers).sort_values("ga:totalEvents", ascending=False).to_excel("add_layers.xlsx")

In [ ]:
query=raw_results[raw_results["ga:eventCategory"]=="Query"]

In [ ]:
query["ga:eventAction"].unique();

In [ ]:
gdf = parser.create_geodataframe(data_frame)
%matplotlib inline

In [ ]:
map_centre=Point(133.3,-26)

australia_bounds = box(110,-45,155,-10)
buff=map_centre.buffer(1)


In [ ]:
#gdf["distance"] = gdf["shape"].centroid.distance(map_centre)
gdf["area"] = gdf["shape"].area
gdf["weighting"] = australia_bounds.area/ gdf["shape"].area * gdf["ga:totalEvents"]
gdf = gdf[gdf["shape"].centroid.intersects(australia_bounds)]
#gdf = gdf[gdf["shape"].centroid.intersects(buff)]

gdf = gdf[gdf["area"] <= australia_bounds.area]
#gdf = gdf[gdf["area"] > 0]
#gdf


x=gdf[gdf['ga:eventAction'] == "Layer:Scanned 250K Geological Map Index"]


In [ ]:
cr.create_spatial_data(gdf)



In [ ]:
x=1